In [1]:
# Celda 1: Carga de Librerías y Datos
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split


In [2]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [3]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [4]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [5]:
train_split, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

In [6]:
# Obtener listas únicas de usuarios e ítems del conjunto de entrenamiento
unique_users = train_split['user'].unique()
unique_items = train_split['item'].unique()

# Crear índices para usuarios e ítems
user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

# Dimensiones de la matriz de ratings
num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 64356
Número de ítems: 146015


In [7]:
from scipy.sparse import csr_matrix

# Crear la matriz dispersa solo con el conjunto de entrenamiento
rows = train_split['user'].map(user_to_index).values
cols = train_split['item'].map(item_to_index).values
ratings = train_split['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (64356, 146015)
Número de ratings no nulos en la matriz: 312280


In [8]:
# Celda 3: Inicialización de Parámetros y Factores Latentes con Bias
# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.02  # Regularización
learning_rate = 0.002  # Tasa de aprendizaje
num_epochs = 50  # Número de iteraciones

# Media global de ratings
mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((64356, 20)), V ((146015, 20))
Sesgos inicializados: bu ((64356,)), bi ((146015,))


In [9]:
# Celda 4: Función de Entrenamiento PMF con Bias
def train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi

In [ ]:
# Celda 5: Entrenamiento del Modelo PMF
print("Iniciando entrenamiento PMF...")
U, V, bu, bi = train_pmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")


Iniciando entrenamiento PMF...
Época 1/50 - Costo total: 1014701.2292
Época 2/50 - Costo total: 964541.8732
Época 3/50 - Costo total: 934966.5709
Época 4/50 - Costo total: 912992.2588
Época 5/50 - Costo total: 895160.9416
Época 6/50 - Costo total: 879979.5402
Época 7/50 - Costo total: 866656.0488
Época 8/50 - Costo total: 854713.3280
Época 9/50 - Costo total: 843838.8148
Época 10/50 - Costo total: 833815.1179
Época 11/50 - Costo total: 824483.6860
Época 12/50 - Costo total: 815723.9468
Época 13/50 - Costo total: 807440.4262
Época 14/50 - Costo total: 799554.2844
Época 15/50 - Costo total: 791997.4445
Época 16/50 - Costo total: 784708.3726
Época 17/50 - Costo total: 777629.1126
Época 18/50 - Costo total: 770703.5952
Época 19/50 - Costo total: 763877.6368
Época 20/50 - Costo total: 757101.3541
Época 21/50 - Costo total: 750334.7130
Época 22/50 - Costo total: 743556.1149
Época 23/50 - Costo total: 736771.8305
Época 24/50 - Costo total: 730021.1815
Época 25/50 - Costo total: 723371.0913
Ép

ValueError: too many values to unpack (expected 2)

In [17]:
# Celda 6: Evaluación del Modelo (MAE)
def calculate_mae(U, V, data):
    total_error = 0
    count = 0
    for _, row in data.iterrows():
        user_id = row['user']
        item_id = row['item']
        rating = row['rating']
        if user_id in user_to_index and item_id in item_to_index:
            pred = np.dot(U[user_to_index[user_id], :], V[item_to_index[item_id], :])
            error = abs(rating - pred)
            total_error += error
            count += 1
    mae = total_error / count if count > 0 else float('inf')
    return mae

# Calcular MAE en el conjunto de validación
mae = calculate_mae(U, V, val_split)
print(f"MAE en el conjunto de validación: {mae:.4f}")

MAE en el conjunto de validación: 3.4185


In [18]:
# Celda 8: Entrenamiento del Modelo SVD
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# Carga de Datos en formato Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader)

# División en conjunto de entrenamiento y validación
trainset, valset = train_test_split(data, test_size=0.2, random_state=42)

# Configuración del modelo SVD
svd = SVD(n_factors=20, n_epochs=50, biased=True, lr_all=0.002, reg_all= 0.02, random_state=42)

print("Entrenando el modelo SVD...")
svd.fit(trainset)
print("Modelo SVD entrenado correctamente.")

Entrenando el modelo SVD...
Modelo SVD entrenado correctamente.


In [ ]:
# Celda 9: Predicción combinada con PMF (con bias) y SVD
def predict_rating(uid, iid, U, V, bu, bi, mu):
    try:
        # Predicción usando SVD
        pred_svd = svd.predict(uid, iid).est
    except:
        pred_svd = None

    # Predicción usando PMF con bias
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]
        pred_pmf = mu + bu[user_idx] + bi[item_idx] + np.dot(U[user_idx, :], V[item_idx, :])
    else:
        pred_pmf = None

    # Promedio de predicciones si ambas son válidas
    if pred_svd is not None and pred_pmf is not None:
        pred = (pred_svd + pred_pmf) / 2
    elif pred_svd is not None:
        pred = pred_svd
    elif pred_pmf is not None:
        pred = pred_pmf
    else:
        pred = mu  # Si ambos fallan, usar la media global

    # Clamping entre 1 y 10 con formato decimal ".0"
    return f"{round(max(1, min(10, pred)))}.0"

In [20]:
# Celda 10: Generación del Archivo de Predicciones
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid, U, V)
    predictions.append((row_id, pred_rating))

# Crear el DataFrame con las predicciones
predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

# Guardar las predicciones en formato CSV
output_filename = "predictions_pmf_svd.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_pmf_svd.csv' generado correctamente.
